In [2]:
import pandas as pd
from transformers import RobertaTokenizer, TrainingArguments
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn import
from datasets import DatasetDict,load_dataset,Dataset
from transformers import Trainer
from transformers import RobertaForSequenceClassification
import torch
from datasets import load_metric
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd
import time
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

#### Reading data

In [3]:
dataset = load_dataset('pandas', data_files='imdb.pkl', split="train")
dataset.remove_columns_(column_names = ['__index_level_0__'])

Using custom data configuration default-8d238810ad5cf425


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset pandas downloaded and prepared to /scratch/lustre/home/vaga6213/.cache/huggingface/datasets/pandas/default-8d238810ad5cf425/0.0.0/6197c1e855b639d75a767140856841a562b7a71d129104973fe1962594877ade. Subsequent calls will reuse this data.


/tmp/slurm-vaga6213-1483/ipykernel_14091/876460931.py:2: FutureWarning: remove_columns_ is deprecated and will be removed in the next major version of datasets. Use Dataset.remove_columns instead.
  dataset.remove_columns_(column_names = ['__index_level_0__'])


#### Preprocessing

In [4]:
dataset = load_dataset('pandas', data_files='imdb.pkl', split="train")
dataset.remove_columns_(column_names = ['__index_level_0__'])
train_testvalid = dataset.train_test_split(test_size=0.1)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
all_dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})
tokenized_datasets = all_dataset.map(tokenize_function, batched=True)
tokenized_datasets

Using custom data configuration default-8d238810ad5cf425
Reusing dataset pandas (/scratch/lustre/home/vaga6213/.cache/huggingface/datasets/pandas/default-8d238810ad5cf425/0.0.0/6197c1e855b639d75a767140856841a562b7a71d129104973fe1962594877ade)


  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'text'],
        num_rows: 2700
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'text'],
        num_rows: 150
    })
    valid: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'text'],
        num_rows: 150
    })
})

#### Training

In [5]:
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels = 2)

training_args = TrainingArguments(
    output_dir='./results'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    tokenizer=tokenizer
)
trainer.train()

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

Step,Training Loss
500,0.433900
1000,0.167100


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1014, training_loss=0.2983719117542696, metrics={'train_runtime': 280.972, 'train_samples_per_second': 28.828, 'train_steps_per_second': 3.609, 'total_flos': 2131199548416000.0, 'train_loss': 0.2983719117542696, 'epoch': 3.0})

#### Saving model and calculating metrics

In [8]:
from datasets import load_metric
from sklearn.metrics import confusion_matrix

In [6]:
trainer.save_model("trained_on_setiment_imbd")

Saving model checkpoint to trained_on_setiment_imbd
Configuration saved in trained_on_setiment_imbd/config.json
Model weights saved in trained_on_setiment_imbd/pytorch_model.bin
tokenizer config file saved in trained_on_setiment_imbd/tokenizer_config.json
Special tokens file saved in trained_on_setiment_imbd/special_tokens_map.json


In [9]:
predictions = trainer.predict(tokenized_datasets["test"])
y_true = tokenized_datasets["test"]["label"]
y_pred = np.argmax(predictions.predictions, axis=-1)
confusion_matrix(y_true, y_pred)

The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 150
  Batch size = 8


array([[71,  3],
       [ 5, 71]])

In [10]:
metric = load_metric("accuracy")
metric.compute(predictions=y_pred, references=y_true)

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

{'accuracy': 0.9466666666666667}